# Yu-Gi-Oh Data loading

In [45]:
import pandas as pd
import requests
import json

In [46]:
response = requests.api.get("https://db.ygoprodeck.com/api/v7/cardinfo.php")

In [47]:
response.status_code

200

In [48]:
with open("allCardData.json", "w") as f:
    json.dump(response.json(), f, indent=4) 

In [ ]:
with open("allCardData.json") as f:
    raw = json.load(f)

card_data = raw["data"]

df = pd.DataFrame(card_data)

In [194]:
df.drop(columns=["ygoprodeck_url", "card_sets", "card_images", "banlist_info"], inplace=True, errors='ignore')
df.head()

,id,name,type,humanReadableCardType,frameType,desc,race,archetype,card_prices,typeline,atk,def,level,attribute,linkval,linkmarkers,pend_desc,monster_desc,scale
0,34541863,"""A"" Cell Breeding Device",Spell Card,Continuous Spell,spell,"During each of your Standby Phases, put 1 A-Co...",Continuous,Alien,"[{'cardmarket_price': '0.15', 'tcgplayer_price...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,64163367,"""A"" Cell Incubator",Spell Card,Continuous Spell,spell,Each time an A-Counter(s) is removed from play...,Continuous,Alien,"[{'cardmarket_price': '0.13', 'tcgplayer_price...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,91231901,"""A"" Cell Recombination Device",Spell Card,Quick-Play Spell,spell,Target 1 face-up monster on the field; send 1 ...,Quick-Play,Alien,"[{'cardmarket_price': '0.08', 'tcgplayer_price...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,73262676,"""A"" Cell Scatter Burst",Spell Card,Quick-Play Spell,spell,"Select 1 face-up ""Alien"" monster you control. ...",Quick-Play,Alien,"[{'cardmarket_price': '0.06', 'tcgplayer_price...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,80181649,"""Case of K9""",Spell Card,Continuous Spell,spell,"When this card is activated: You can add 1 ""K9...",Continuous,K9,"[{'cardmarket_price': '1.13', 'tcgplayer_price...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [195]:
df['price_dict'] = df['card_prices'].apply(lambda x: x[0] if isinstance(x, list) and x else {})

prices_df = pd.json_normalize(df['price_dict'])

df = pd.concat([df.drop(columns=['card_prices', 'price_dict']), prices_df], axis=1)

price_cols = prices_df.columns
df[price_cols] = df[price_cols].astype(float)

In [196]:
df.head()

,id,name,type,humanReadableCardType,frameType,desc,race,archetype,typeline,atk,...,linkval,linkmarkers,pend_desc,monster_desc,scale,cardmarket_price,tcgplayer_price,ebay_price,amazon_price,coolstuffinc_price
0,34541863,"""A"" Cell Breeding Device",Spell Card,Continuous Spell,spell,"During each of your Standby Phases, put 1 A-Co...",Continuous,Alien,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.15,0.23,0.99,24.45,0.25
1,64163367,"""A"" Cell Incubator",Spell Card,Continuous Spell,spell,Each time an A-Counter(s) is removed from play...,Continuous,Alien,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.13,0.23,1.25,0.50,0.25
2,91231901,"""A"" Cell Recombination Device",Spell Card,Quick-Play Spell,spell,Target 1 face-up monster on the field; send 1 ...,Quick-Play,Alien,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.08,0.26,0.99,0.50,0.49
3,73262676,"""A"" Cell Scatter Burst",Spell Card,Quick-Play Spell,spell,"Select 1 face-up ""Alien"" monster you control. ...",Quick-Play,Alien,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.06,0.15,2.00,9.76,0.25
4,80181649,"""Case of K9""",Spell Card,Continuous Spell,spell,"When this card is activated: You can add 1 ""K9...",Continuous,K9,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.13,0.00,0.00,0.00,0.00


In [197]:
df_spell_cards = df[df["type"] == "Spell Card"].copy()
df_spell_cards.drop(columns=["atk", "def", "level", "race", "attribute"], inplace=True, errors='ignore')

In [198]:
df_trap_cards = df[df["type"] == "Trap Card"].copy()
df_trap_cards.drop(columns=["atk", "def", "level", "race", "attribute"], inplace=True, errors='ignore')

In [199]:
df_monster_cards = df[(df["type"] != "Spell Card") & (df["type"] != "Trap Card")].copy()

### Checking that we have the same number of cards in the tables

In [200]:
df_spell_cards.shape[0] + df_monster_cards.shape[0] + df_trap_cards.shape[0]

13787

In [201]:
df.shape[0]

13787

### Checking if any columns are always NaN so they can be removed

In [202]:
df_monster_cards.isna().all()

id                       False
name                     False
type                     False
humanReadableCardType    False
frameType                False
desc                     False
race                     False
archetype                False
typeline                 False
atk                      False
def                      False
level                    False
attribute                False
linkval                  False
linkmarkers              False
pend_desc                False
monster_desc             False
scale                    False
cardmarket_price         False
tcgplayer_price          False
ebay_price               False
amazon_price             False
coolstuffinc_price       False
dtype: bool

In [203]:
df_spell_cards.isna().all()

id                       False
name                     False
type                     False
humanReadableCardType    False
frameType                False
desc                     False
archetype                False
typeline                  True
linkval                   True
linkmarkers               True
pend_desc                 True
monster_desc              True
scale                     True
cardmarket_price         False
tcgplayer_price          False
ebay_price               False
amazon_price             False
coolstuffinc_price       False
dtype: bool

#### Spell and Trap cards are the same in this regard and so both have these columns dropped

In [204]:
df_spell_cards.drop(columns=["typeline","linkval", "linkmarkers", "pend_desc", "monster_desc", "scale"], inplace=True, errors='ignore')
df_trap_cards.drop(columns=["typeline","linkval", "linkmarkers", "pend_desc", "monster_desc", "scale"], inplace=True, errors='ignore')

## Dataframe to CSV

In [209]:
df_monster_cards.to_csv("Monster_Cards.csv", index=False)
df_spell_cards.to_csv("Spell_Cards.csv", index=False)
df_trap_cards.to_csv("Trap_Cards.csv", index=False)